##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Fundamentos de TensorFlow

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/basics"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/guide/basics.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/guide/basics.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver código fuente en GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/guide/basics.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar bloc de notas</a>
</td>
</table>

Esta guía ofrece una rápida visión general de los conceptos básicos de *TensorFlow*. Cada apartado de este documento es una visión general de un tema más amplio. Puede encontrar enlaces a guías completas al final de cada apartado.

TensorFlow es una plataforma integral para el aprendizaje automático. Es compatible con lo siguiente:

- Cálculo numérico basado en arreglos multidimensionales (similar a <a href="https://numpy.org/" class="external">NumPy</a>).
- GPU y procesamiento distribuido
- Diferenciación automática
- Construcción, entrenamiento y exportación de modelos
- Y más

## Tensores

TensorFlow opera sobre arreglos multidimensionales o *tensores* representados como objetos `tf.Tensor`. Aquí se muestra un tensor bidimensional:

In [ ]:
import tensorflow as tf

x = tf.constant([[1., 2., 3.],
                 [4., 5., 6.]])

print(x)
print(x.shape)
print(x.dtype)

Los atributos más importantes de un `tensor` son su `shape` y su `dtype`:

- `Tensor.shape`: indica el tamaño del tensor a lo largo de cada uno de sus ejes.
- `Tensor.dtype`: le indica el tipo de todos los elementos del tensor.

TensorFlow implementa operaciones matemáticas estándares sobre tensores, así como muchas operaciones especializadas para el aprendizaje automático.

Por ejemplo:

In [ ]:
x + x

In [ ]:
5 * x

In [ ]:
x @ tf.transpose(x)

In [ ]:
tf.concat([x, x, x], axis=0)

In [ ]:
tf.nn.softmax(x, axis=-1)

In [ ]:
tf.reduce_sum(x)

Nota: Típicamente, en cualquier lugar donde una función TensorFlow espere un `Tensor` como entrada, la función también aceptará cualquier cosa que pueda ser convertida a un `Tensor` usando `tf.convert_to_tensor`. Vea a continuación un ejemplo.

In [ ]:
tf.convert_to_tensor([1,2,3])

In [ ]:
tf.reduce_sum([1,2,3])

Puede resultar lento ejecutar grandes cálculos en la CPU. Cuando se configura correctamente, TensorFlow puede usar hardware acelerador como las GPU para ejecutar operaciones muy rápidamente.

In [ ]:
if tf.config.list_physical_devices('GPU'):
  print("TensorFlow **IS** using the GPU")
else:
  print("TensorFlow **IS NOT** using the GPU")

Consulte la [Guía del tensor](tensor.ipynb) para más detalles.

## Variables

Los objetos `tf.Tensor` normales son inmutables. Para almacenar las ponderaciones del modelo (u otro estado mutable) en TensorFlow use un `tf.Variable`.

In [ ]:
var = tf.Variable([0.0, 0.0, 0.0])

In [ ]:
var.assign([1, 2, 3])

In [ ]:
var.assign_add([1, 1, 1])

Consulte la [guía de Variables](variable.ipynb) para más detalles.

## Diferenciación automática

<a href="https://en.wikipedia.org/wiki/Gradient_descent" class="external"><em>El descenso gradiente</em></a> y los algoritmos relacionados son una piedra angular del aprendizaje automático moderno.

Para ello, TensorFlow implementa la diferenciación automática (autodiff), que usa el cálculo para computar gradientes. Normalmente lo usará para calcular el gradiente del *error* o *pérdida* de un modelo con respecto a sus ponderaciones.

In [ ]:
x = tf.Variable(1.0)

def f(x):
  y = x**2 + 2*x - 5
  return y

In [ ]:
f(x)

En `x = 1.0`, `y = f(x) = (1**2 + 2*1 - 5) = -2`.

La derivada de `y` es `y' = f'(x) = (2*x + 2) = 4`. TensorFlow puede calcular esto automáticamente:

In [ ]:
with tf.GradientTape() as tape:
  y = f(x)

g_x = tape.gradient(y, x)  # g(x) = dy/dx

g_x

Este ejemplo simplificado sólo toma la derivada con respecto a un único escalar (`x`), pero TensorFlow puede calcular el gradiente con respecto a cualquier número de tensores no escalares simultáneamente.

Consulte la guía [Autodiff](autodiff.ipynb) para más detalles.

## Gráficos y tf.function

Aunque puede usar TensorFlow de forma interactiva como cualquier librería de Python, TensorFlow también proporciona herramientas para:

- **Optimización del rendimiento**: para acelerar el entrenamiento y la inferencia.
- **Exportar**: para que pueda guardar su modelo cuando haya terminado el entrenamiento.

Estos requieren que usted use `tf.function` para separar su código de TensorFlow puro de Python.

In [ ]:
@tf.function
def my_func(x):
  print('Tracing.\n')
  return tf.reduce_sum(x)

La primera vez que se ejecuta la función `tf.function`, aunque se ejecuta en Python, captura un grafo completo y optimizado que representa los cálculos TensorFlow realizados dentro de la función.

In [ ]:
x = tf.constant([1, 2, 3])
my_func(x)

En llamadas posteriores, TensorFlow sólo ejecuta el grafo optimizado, saltándose cualquier paso que no sea de TensorFlow. A continuación, observe que `my_func` no imprime *tracing* ya que `print` es una función Python, no una función TensorFlow.

In [ ]:
x = tf.constant([10, 9, 8])
my_func(x)

Un grafo puede no ser reutilizable para entradas con una *firma* diferente (`shape` y `dtype`), por lo que en su lugar se genera un nuevo grafo:

In [ ]:
x = tf.constant([10.0, 9.1, 8.2], dtype=tf.float32)
my_func(x)

Estos gráficos capturados proporcionan dos beneficios:

- En muchos casos suponen una importante aceleración en la ejecución (aunque no en este ejemplo trivial).
- Puede exportar estos grafos, utilizando `tf.saved_model`, para ejecutarlos en otros sistemas como un [servidor](https://www.tensorflow.org/tfx/serving/docker) o un [dispositivo móvil](https://www.tensorflow.org/lite/guide), sin necesidad de instalar Python.

Consulte [Introducción a los grafos](intro_to_graphs.ipynb) para más detalles.

## Módulos, capas y modelos

`tf.Module` es una clase para administrar sus objetos `tf.Variable`, y los objetos `tf.function` que operan sobre ellos. La clase `tf.Module` es necesaria para soportar dos características significativas:

1. Puede guardar y restaurar los valores de sus variables usando `tf.train.Checkpoint`. Esto es útil durante el entrenamiento, ya que permite guardar y restaurar rápidamente el estado de un modelo.
2. Puede importar y exportar los valores `tf.Variable` *y* los grafos `tf.function` usando `tf.saved_model`. Esto permite ejecutar su modelo independientemente del programa Python que lo creó.

He aquí un ejemplo completo que exporta un simple objeto `tf.Module`:

In [ ]:
class MyModule(tf.Module):
  def __init__(self, value):
    self.weight = tf.Variable(value)

  @tf.function
  def multiply(self, x):
    return x * self.weight

In [ ]:
mod = MyModule(3)
mod.multiply(tf.constant([1, 2, 3]))

Guarde el `Module`:

In [ ]:
save_path = './saved'
tf.saved_model.save(mod, save_path)

El SavedModel resultante es independiente del código que lo creó. Puede cargar un SavedModel desde Python, otras vinculaciones de lenguaje o [TensorFlow Serving](https://www.tensorflow.org/tfx/serving/docker). También puede convertirlo para que funcione con [TensorFlow Lite](https://www.tensorflow.org/lite/guide) o [TensorFlow JS](https://www.tensorflow.org/js/guide).

In [ ]:
reloaded = tf.saved_model.load(save_path)
reloaded.multiply(tf.constant([1, 2, 3]))

Las clases `tf.keras.layers.Layer` y `tf.keras.Model` se basan en `tf.Module` proporcionando funcionalidad adicional y métodos convenientes para construir, entrenar y guardar modelos. Algunos de ellos se demuestran en la siguiente sección.

Consulte [Introducción a los módulos](intro_to_modules.ipynb) para más detalles.

## Bucles de entrenamiento

Ahora reúna todo esto para construir un modelo básico y entrénelo desde cero.

En primer lugar, cree algunos datos de ejemplo. Esto genera una nube de puntos que sigue vagamente una curva cuadrática:

In [ ]:
import matplotlib
from matplotlib import pyplot as plt

matplotlib.rcParams['figure.figsize'] = [9, 6]

In [ ]:
x = tf.linspace(-2, 2, 201)
x = tf.cast(x, tf.float32)

def f(x):
  y = x**2 + 2*x - 5
  return y

y = f(x) + tf.random.normal(shape=[201])

plt.plot(x.numpy(), y.numpy(), '.', label='Data')
plt.plot(x, f(x), label='Ground truth')
plt.legend();

Cree un modelo cuadrático con ponderaciones inicializadas aleatoriamente y un sesgo:

In [ ]:
class Model(tf.Module):

  def __init__(self):
    # Randomly generate weight and bias terms
    rand_init = tf.random.uniform(shape=[3], minval=0., maxval=5., seed=22)
    # Initialize model parameters
    self.w_q = tf.Variable(rand_init[0])
    self.w_l = tf.Variable(rand_init[1])
    self.b = tf.Variable(rand_init[2])
  
  @tf.function
  def __call__(self, x):
    # Quadratic Model : quadratic_weight * x^2 + linear_weight * x + bias
    return self.w_q * (x**2) + self.w_l * x + self.b

En primer lugar, observe el rendimiento de su modelo antes del entrenamiento:

In [ ]:
quad_model = Model()

In [ ]:
def plot_preds(x, y, f, model, title):
  plt.figure()
  plt.plot(x, y, '.', label='Data')
  plt.plot(x, f(x), label='Ground truth')
  plt.plot(x, model(x), label='Predictions')
  plt.title(title)
  plt.legend()

In [ ]:
plot_preds(x, y, f, quad_model, 'Before training')

Ahora, defina una pérdida para su modelo:

Dado que este modelo pretende predecir valores continuos, el error cuadrático medio (MSE) es una buena elección para la función de pérdida. Dado un vector de predicciones, $\hat{y}$, y un vector de objetivos verdaderos, $y$, el MSE se define como la media de las diferencias al cuadrado entre los valores predichos y la verdad sobre el terreno.

$MSE = \frac{1}{m}\sum_{i=1}^{m}(\hat{y}_i -y_i)^2$

In [ ]:
def mse_loss(y_pred, y):
  return tf.reduce_mean(tf.square(y_pred - y))

Escriba un bucle de entrenamiento básico para el modelo. El bucle usará la función de pérdida MSE y sus gradientes con respecto a la entrada para actualizar iterativamente los parámetros del modelo. Usar minilotes para el entrenamiento le ofrece tanto eficiencia de memoria como una convergencia más rápida. La API `tf.data.Dataset` tiene funciones útiles para agrupar por lotes y barajar.

In [ ]:
batch_size = 32
dataset = tf.data.Dataset.from_tensor_slices((x, y))
dataset = dataset.shuffle(buffer_size=x.shape[0]).batch(batch_size)

In [ ]:
# Set training parameters
epochs = 100
learning_rate = 0.01
losses = []

# Format training loop
for epoch in range(epochs):
  for x_batch, y_batch in dataset:
    with tf.GradientTape() as tape:
      batch_loss = mse_loss(quad_model(x_batch), y_batch)
    # Update parameters with respect to the gradient calculations
    grads = tape.gradient(batch_loss, quad_model.variables)
    for g,v in zip(grads, quad_model.variables):
        v.assign_sub(learning_rate*g)
  # Keep track of model loss per epoch
  loss = mse_loss(quad_model(x), y)
  losses.append(loss)
  if epoch % 10 == 0:
    print(f'Mean squared error for step {epoch}: {loss.numpy():0.3f}')

# Plot model results
print("\n")
plt.plot(range(epochs), losses)
plt.xlabel("Epoch")
plt.ylabel("Mean Squared Error (MSE)")
plt.title('MSE loss vs training iterations');

Ahora, observe el rendimiento de su modelo después del entrenamiento:

In [ ]:
plot_preds(x, y, f, quad_model, 'After training')

Eso funciona, pero recuerde que las implementaciones de las utilidades de entrenamiento comunes están disponibles en el módulo `tf.keras`. Por lo tanto, considere usarlas antes de escribir las suyas propias. Para empezar, los métodos `Model.compile` y `Model.fit` implementan un bucle de entrenamiento para usted:

Comience por crear un modelo Sequential en Keras usando `tf.keras.Sequential`. Una de las capas Keras más sencillas es la capa dense, que puede instanciarse con `tf.keras.layers.Dense`. La capa Dense es capaz de aprender relaciones lineales multidimensionales de la forma $\mathrm{Y} = \mathrm{W}\mathrm{X} + \vec{b}$. Para aprender una ecuación no lineal de la forma, $w_1x^2 + w_2x + b$, la entrada de la capa densa debe ser una matriz de datos con $x^2$ y $x$ como características. La capa lambda, `tf.keras.layers.Lambda`, puede usarse para realizar esta transformación de apilamiento.

In [ ]:
new_model = tf.keras.Sequential([
    tf.keras.layers.Lambda(lambda x: tf.stack([x, x**2], axis=1)),
    tf.keras.layers.Dense(units=1, kernel_initializer=tf.random.normal)])

In [ ]:
new_model.compile(
    loss=tf.keras.losses.MSE,
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.01))

history = new_model.fit(x, y,
                        epochs=100,
                        batch_size=32,
                        verbose=0)

new_model.save('./my_new_model')

Observe el rendimiento de su modelo Keras después del entrenamiento:

In [ ]:
plt.plot(history.history['loss'])
plt.xlabel('Epoch')
plt.ylim([0, max(plt.ylim())])
plt.ylabel('Loss [Mean Squared Error]')
plt.title('Keras training progress');

In [ ]:
plot_preds(x, y, f, new_model, 'After Training: Keras')

Consulte [Bucles básicos de entrenamiento](basic_training_loops.ipynb) y la guía [Keras](https://www.tensorflow.org/guide/keras) para más detalles.